# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 47 new papers today
          16 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/15 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.14233


extracting tarball to tmp_2210.14233...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.14237


extracting tarball to tmp_2210.14237... done.
Retrieving document from  https://arxiv.org/e-print/2210.14239


...

extracting tarball to tmp_2210.14817... done.
Retrieving document from  https://arxiv.org/e-print/2210.14853


extracting tarball to tmp_2210.14853...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.14890


extracting tarball to tmp_2210.14890...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14233-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14233) | **ELVES IV: The Satellite Stellar-to-Halo Mass Relation Beyond the  Milky-Way**  |
|| Shany Danieli, et al. -- incl., <mark>Jenny E. Greene</mark>, <mark>Andy D. Goulding</mark> |
|*Appeared on*| *2022-10-27*|
|*Comments*| *Submitted to ApJ, comments are very welcome!*|
|**Abstract**| Quantifying the connection between galaxies and their host dark matter halos has been key for testing cosmological models on various scales. Below $M_\star \sim 10^9\,M_\odot$, such studies have primarily relied on the satellite galaxy population orbiting the Milky Way. Here we present new constraints on the connection between satellite galaxies and their host dark matter subhalos using the largest sample of satellite galaxies in the Local Volume ($D \lesssim 12\,\mathrm{Mpc}$) to date. We use $250$ confirmed and $71$ candidate dwarf satellites around 27 Milky Way (MW)-like hosts from the Exploration of Local VolumE Satellites (ELVES) Survey and use the semi-analytical SatGen model for predicting the population of dark matter subhalos expected in the same volume. Through a Bayesian model comparison of the observed and the forward-modeled satellite stellar mass functions (SSMF), we infer the satellite stellar-to-halo mass relation. We find that the observed SSMF is best reproduced when subhalos are populated by a relation of the form $M_\star \propto M^\alpha_\mathrm{peak}$, with a moderate slope of $\alpha=2.0 \pm 0.1$, and a scatter that grows with decreasing $M_\mathrm{peak}$. We find a significantly larger scatter towards lower peak halo masses, compared to prior studies that relied mainly on MW satellites. We conclude that this scatter results from a combination of star formation stochasticity and host-to-host scatter. Our new model for the satellite-subhalo connection has important implications for both of these baryonic-impacted effects, as well as on dark matter physics. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14237-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14237) | **ELVES III: Environmental Quenching by Milky Way-Mass Hosts**  |
|| <mark>Jenny E. Greene</mark>, et al. |
|*Appeared on*| *2022-10-27*|
|*Comments*| *18 pages, 12 figures, submitted to ApJ, comments welcome*|
|**Abstract**| Isolated dwarf galaxies nearly always exhibit robust star formation but satellite dwarf galaxies are often devoid of young stars, even in Milky Way-mass groups. Dwarf galaxies thus offer an important laboratory into the environmental processes that cease star formation. We explore the balance of quiescent and star-forming galaxies (quenched fractions) for a sample of ~400 satellite galaxies around 30 Local Volume hosts from the Exploration of Local VolumE Satellites (ELVES) Survey. We present quenched fractions as a function of satellite stellar mass, projected radius, and host halo mass, to conclude that overall, the quenched fractions are similar to the Milky Way, dropping below 50\% at satellite M*~10^8 Msun. There is a ~10% difference in quenched fraction for satellites within and outside of ~R_vir/2 at fixed satellite mass. Finally, the satellite mass at which quenching grows inefficient grows steadily with host halo mass. Through comparison with the semi-analytic modeling code SatGen, we are also able to infer average quenching times as a function of satellite mass in host halo-mass bins. There is a gradual increase in quenching time with satellite stellar mass rather than the abrupt change from rapid to slow quenching that has been inferred for the Milky Way. We also generally infer longer average quenching times than recent hydrodynamical simulations. Our results vindicate models that suggest a wide range of quenching times are possible via ram pressure stripping, depending on the clumpiness of the circumgalactic medium, the orbits of the satellites, and the degree of earlier preprocessing. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14239-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14239) | **Precision Redshift-Space Galaxy Power Spectra using Zel'dovich Control  Variates**  |
|| <mark>Joseph DeRose</mark>, Shi-Fan Chen, Nickolas Kokron, Martin White |
|*Appeared on*| *2022-10-27*|
|*Comments*| *17 pages main text, 5 pages of appendices, 9 figures*|
|**Abstract**| Numerical simulations in cosmology require trade-offs between volume, resolution and run-time that limit the volume of the Universe that can be simulated, leading to sample variance in predictions of ensemble-average quantities such as the power spectrum or correlation function(s). Sample variance is particularly acute at large scales, which is also where analytic techniques can be highly reliable. This provides an opportunity to combine analytic and numerical techniques in a principled way to improve the dynamic range and reliability of predictions for clustering statistics. In this paper we extend the technique of Zel'dovich control variates, previously demonstrated for 2-point functions in real space, to reduce the sample variance in measurements of 2-point statistics of biased tracers in redshift space. We demonstrate that with this technique, we can reduce the sample variance of these statistics down to their shot-noise limit out to $k \sim 0.2\, h\rm Mpc^{-1}$. This allows a better matching with perturbative models and improved predictions for the clustering of e.g.~quasars, galaxies and neutral Hydrogen measured in spectroscopic redshift surveys at very modest computational expense. We discuss the implementation of ZCV, give some examples and provide forecasts for the efficacy of the method under various conditions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14817-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14817) | **Joint photon-electron Lorentz violation parameter plane from LHAASO data**  |
|| <mark>Ping He</mark>, Bo-Qiang Ma |
|*Appeared on*| *2022-10-27*|
|*Comments*| *8 latex pages, 3 figures, version for journal publication*|
|**Abstract**| The Large High Altitude Air Shower Observatory~(LHAASO) is one of the most sensitive gamma-ray detector arrays, whose ultrahigh-energy~(UHE) work bands not only help to study the origin and acceleration mechanism of UHE cosmic rays, but also provide the opportunity to test fundamental physics concepts such as Lorentz symmetry. LHAASO directly observes the $1.42~\mathrm{PeV}$ highest-energy photon. By adopting the synchrotion self-Compton model LHAASO also suggests that the $1.12~\mathrm{PeV}$ high-energy photon from Crab Nebula corresponds to a $2.3~\mathrm{PeV}$ high-energy electron. We study the $1.42~\mathrm{PeV}$ photon decay and the $2.3~\mathrm{PeV}$ electron decay to perform a joint analysis on photon and electron two-dimensional Lorentz violation~(LV) parameter plane. Our analysis is systematic and comprehensive, and we naturally get the strictest constraints from merely considering photon LV effect in photon decay and electron LV effect in electron decay. Our result also permits the parameter space for new physics beyond relativity. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14890-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14890) | **The deconvolved distribution estimator: enhancing reionisation-era CO  line-intensity mapping analyses with a cross-correlation analogue for  one-point statistics**  |
|| Dongwoo T. Chung, et al. -- incl., <mark>Liju Philip</mark> |
|*Appeared on*| *2022-10-27*|
|*Comments*| *10 pages + acknowledgements/bibliography/appendix (12 pages total); 10 figures, 2 tables; to be submitted to MNRAS*|
|**Abstract**| We present the deconvolved distribution estimator (DDE), an extension of the voxel intensity distribution (VID), in the context of future observations proposed as part of the CO Mapping Array Project (COMAP). The DDE exploits the fact that the observed VID is a convolution of correlated signal intensity distributions and uncorrelated noise or interloper intensity distributions. By deconvolving the individual VID of two observables away from their joint VID in a Fourier-space operation, the DDE suppresses sensitivity to interloper emission while maintaining sensitivity to correlated components. The DDE thus improves upon the VID by rejecting uncorrelated noise and interloper biases, which is useful in the context of COMAP observations that observe different rotational transitions of CO from the same comoving volume in different observing frequency bands. Fisher forecasts suggest that the theoretical sensitivity in the DDE allows significant improvements in constraining power compared to either the cross power spectrum or the individual VID data, and matches the constraining power of the combination of all other one- and two-point summary statistics. Future work should further investigate the covariance and model-dependent behaviour of this novel one-point cross-correlation statistic. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14853-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14853) | **A family of potential-density pairs for galactic bars**  |
|| <mark>Walter Dehnen</mark>, Hossam Aly |
|*Appeared on*| *2022-10-27*|
|*Comments*| *10 pages, accepted for publication at MNRAS*|
|**Abstract**| We present a family of analytical potential-density pairs for barred discs, which can be combined to describe galactic bars in a realistic way, including boxy/peanut components. We illustrate this with two reasonable compound models. Computer code for the evaluation of potential, forces, density, and projected density is freely provided. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14244-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14244) | **Bars and boxy/peanut bulges in thin and thick discs. II. Can bars form  in hot thick discs?**  |
|| <mark>Soumavo Ghosh</mark>, Francesca Fragkoudi, Paola Di Matteo, Kanak Saha |
|*Appeared on*| *2022-10-27*|
|*Comments*| *20 pages, 14 figures, 1 table (including appendix). Comments are welcome!*|
|**Abstract**| The Milky Way as well as external galaxies possess a thick disc. However, the dynamical role of the (geometrically) thick disc on the bar formation and evolution is not fully understood. Here, we investigate the effect of thick discs in bar formation and evolution by means of a suite of $N$-body models of (kinematically cold) thin-(kinematically hot) thick discs. We systematically vary the mass fraction of the thick disc, the thin-to-thick disc scale length ratio as well as thick disc's scale height to examine the bar formation under diverse dynamical scenarios. Bars form almost always in our models, even in presence of a massive thick disc. The bar in the thick disc closely follow the overall growth and temporal evolution of the thin disc's bar, only the bar in the thick disc is weaker than the bar in the thin disc. The formation of stronger bars is associated with a simultaneous larger loss of angular momentum and a larger radial heating. In addition, we demonstrate a preferential loss of angular momentum and a preferential radial heating of disc stars, along the azimuthal direction within the extent of the bar, in both thin and thick disc stars. For purely thick disc models (without any thin disc), the bar formation critically depends on the disc scale length and scale height. A larger scale length and/or a larger vertical scale height delays the bar formation time and/or suppresses the bar formation almost completely in thick-disc-only models. We find that the Ostriker-Peeble criterion predicts the bar instability scenarios in our models better than the Efstathiou-Lake-Negroponte criterion. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14293-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14293) | **Community Report from the Biosignatures Standards of Evidence Workshop**  |
|| Victoria Meadows, et al. -- incl., <mark>Heather Graham</mark> |
|*Appeared on*| *2022-10-27*|
|*Comments*| *86 pages, 14 figures, workshop report*|
|**Abstract**| The search for life beyond the Earth is the overarching goal of the NASA Astrobiology Program, and it underpins the science of missions that explore the environments of Solar System planets and exoplanets. However, the detection of extraterrestrial life, in our Solar System and beyond, is sufficiently challenging that it is likely that multiple measurements and approaches, spanning disciplines and missions, will be needed to make a convincing claim. Life detection will therefore not be an instantaneous process, and it is unlikely to be unambiguous-yet it is a high-stakes scientific achievement that will garner an enormous amount of public interest. Current and upcoming research efforts and missions aimed at detecting past and extant life could be supported by a consensus framework to plan for, assess and discuss life detection claims (c.f. Green et al., 2021). Such a framework could help increase the robustness of biosignature detection and interpretation, and improve communication with the scientific community and the public. In response to this need, and the call to the community to develop a confidence scale for standards of evidence for biosignature detection (Green et al., 2021), a community-organized workshop was held on July 19-22, 2021. The meeting was designed in a fully virtual (flipped) format. Preparatory materials including readings, instructional videos and activities were made available prior to the workshop, allowing the workshop schedule to be fully dedicated to active community discussion and prompted writing sessions. To maximize global interaction, the discussion components of the workshop were held during business hours in three different time zones, Asia/Pacific, European and US, with daily information hand-off between group organizers. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

73  publications in the last 7 days.
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers